In [10]:
import pandas as pd
from pathlib import Path
% load_ext autoreload
% autoreload 2

path_root = Path().absolute().parent.resolve()

In [ ]:
def lag_var(df: pd.DataFrame, columns: list, lag: int = 1) -> pd.DataFrame:
    """
    lag the Series `lag` times
    df: dataframe to create lags in
    columns: list of columns, where this should be applied to
    !warning! drops na columns, which shortens df
    """
    for col in columns:
        df = pd.concat([df] + [df[col].shift(lag).rename(f'{df[col].name}_{lag + 1}') for lag in range(lag)],
                       axis=1)
    return df.dropna()


def filter_non_log_vals_from_df(df):
    # filter non-log values so they aren't duplicated in the data
    drop_list = [col.replace("_log", "") for col in df.loc[:, df.columns.str.contains("log")].columns]
    df.drop(drop_list, axis=1, inplace=True)
    return df

In [ ]:
def simple_split(df):
    """
    manual split: train: 2010-2017, validation: 2018, test: 2019-2020 (because of corona)
    (73/9/18 or 8y/1y/2y)
    """
    splits = {
        "train": slice("2010", "2016"),
        "val": slice("2017", "2018"),
        "test": slice("2019", None)
    }
    return splits

In [26]:
class LaggedLogStockDataLoader:
    """
    simplified version of the Data Loader for jupyter notebooks -> function calls got flattened
    This data_loader converts the raw dataset into a train-val-test split, by first cleaning the data
    dropping irrelevant columns and empty rows and then lagging necessary parameters.
    """

    def __init__(self, path_data=None, drop_na=True, split_fn=simple_split, load=False):
        self.df = None
        self.y_columns = ["outcome_rv", "outcome_rv_log"]
        try:
            self.splits = split_fn(self.df)
        except Exception as e:
            raise Exception("Split not implemented") from e

        if path_data:
            self.path_data = path_data
        else:
            self.path_data = path_root / "resources" / "raw" / "BAC.csv"
        if load:
            # load the data from a hdf5 instead of initializing it from a csv
            self.load()
        else:
            # try to load a csv and process it
            self.from_csv(drop_na, split_fn)

    def from_csv(self, drop_na, split_fn):
        self.df = pd.read_csv(self.path_data, parse_dates=["DT"])
        # data cleanup - regularize lag_rv_day to lag_rv_day_1
        self.df.rename(columns={"lag_rv_day": "lag_rv_day_1", "lag_rv_day_log": "lag_rv_day_log_1"}, inplace=True)
        # data cleanup - drop rf because the data is broken and 3month, because it has too many Na values
        self.df.drop(["rf", "lag_rv_3month", "lag_rv_3month_log"], axis=1, inplace=True)
        # set index to DT to get datetimeindexes (required for simple splits) and remove the variable from the table
        self.df.set_index("DT", inplace=True)
        # drop na values
        self.df = filter_non_log_vals_from_df(self.df)
        if drop_na:
            self.df = self.df.dropna()
        # lag relevant parameters
        self.df = lag_var(self.df, ["lag_rv_week_log", "lag_rv_month_log"], 21)

    def _get_x(self, df):
        """
        return passed df with only X columns
        """
        return df[df.columns.difference(self.y_columns)]

    def _get_y(self, df):
        """
        return passed df with only Y columns
        """
        return df[[x for x in self.y_columns if x in df]]

    def _get_data(self, data_name):
        """
        :param data_name: name of split to get "train", "val" or "test"
        :return Tuple: the dataset as a X,Y tuple
        """
        return self._get_x(self.df[self.splits[data_name]]), self._get_y(self.df[self.splits[data_name]])

    def get_train_data(self):
        return self._get_data("train")

    def get_validation_data(self):
        return self._get_data("val")

    def get_test_data(self):
        return self._get_data("test")

    def get_labels(self):
        return [x for x in self.df.columns]

    def get_df(self):
        return self.df

    def save(self, path_save):
        """
        save the data split as hdf5
        :param Path path_save: where to save the hdf5 (along with the file name
        :return Bool success
        """
        hdf = path_save
        # since we will be using lots of jupyter notebooks, it will simplify our work to redundantly save the datasplits, instead of importing the dataloader (which would be the better solution)
        x_train, y_train = self.get_train_data()
        x_val, y_val = self.get_validation_data()
        x_test, y_test = self.get_test_data()
        self.df.to_hdf(hdf, key="df")
        x_train.to_hdf(hdf, key="x_train")
        x_val.to_hdf(hdf, key="x_val")
        x_test.to_hdf(hdf, key="x_test")
        y_train.to_hdf(hdf, key="y_train")
        y_val.to_hdf(hdf, key="y_val")
        y_test.to_hdf(hdf, key="y_test")

    def load(self):
        """
        load the datasets from a hdf5 file
        :return Bool: success
        """
        with pd.HDFStore(self.path_data.as_posix()) as store:
            self.df = store["df"]


data_loader = LaggedLogStockDataLoader()

Now we can load the data using the data_loader and easily access the splits

In [21]:
data_train = data_loader.get_train_data()  # returns (X,Y) for training
data_val = data_loader.get_validation_data()  # returns (X,Y) for validation
data_test = data_loader.get_test_data()  # returns (X,Y) for test

If we want to skip this step later or work on the data without having to get the dataloader, we can use `data_loader.save()` to store the processed dataframe in a hdf5 store (only df is stored, not the splits)

In [27]:
path_to_save = path_root / "resources" / "formatted" / "data_formatted.h5"
data_loader.save(path_to_save)

They can be retrieved by the data_loader as well, which gives easy access to the get_X_data() methods

In [25]:
data_loader = LaggedLogStockDataLoader(path_to_save, load=True)
data_loader.get_train_data()[0]

,hml,lag_rv_day_log_1,lag_rv_day_log_10,lag_rv_day_log_11,lag_rv_day_log_12,lag_rv_day_log_13,lag_rv_day_log_14,lag_rv_day_log_15,lag_rv_day_log_16,lag_rv_day_log_17,...,lag_rv_week_log_9,liquidity_day_log,liquidity_month_log,liquidity_week_log,mktMinusRiskfree,return_sign_day,return_sign_month,return_sign_week,smb,vix_log
DT,,,,,,,,,,,,,,,,,,,,,
2010-03-04,0.29,-9.253527,-8.762848,-7.970345,-8.109983,-8.220418,-8.100960,-8.246101,-7.744577,-7.212612,...,-8.331060,10.310252,10.760061,10.571604,0.15,0,1.0,0.0,0.09,2.929592
2010-03-05,-0.04,-9.057764,-8.591706,-8.762848,-7.970345,-8.109983,-8.220418,-8.100960,-8.246101,-7.744577,...,-8.258314,10.580429,10.760227,10.497638,0.30,1,1.0,0.0,0.07,2.857619
2010-03-08,0.35,-9.311731,-7.856689,-8.591706,-8.762848,-7.970345,-8.109983,-8.220418,-8.100960,-8.246101,...,-8.306274,10.294820,10.751980,10.444478,1.42,0,1.0,0.0,0.39,2.878637
2010-03-09,0.08,-8.644398,-8.349784,-7.856689,-8.591706,-8.762848,-7.970345,-8.109983,-8.220418,-8.100960,...,-8.408563,10.598508,10.727887,10.448592,0.03,1,1.0,1.0,0.10,2.885917
2010-03-10,-0.18,-8.272543,-8.481787,-8.349784,-7.856689,-8.591706,-8.762848,-7.970345,-8.109983,-8.220418,...,-8.396254,11.107075,10.725954,10.578217,0.13,1,1.0,1.0,0.29,2.921547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-23,0.37,-9.967327,-8.380038,-8.828062,-8.184177,-8.752170,-8.728730,-8.771433,-8.130522,-8.136835,...,-8.498381,9.982299,10.786081,10.373212,-0.31,1,1.0,0.0,-0.77,2.437116
2016-12-27,0.02,-10.189039,-8.347459,-8.380038,-8.828062,-8.184177,-8.752170,-8.728730,-8.771433,-8.130522,...,-8.111388,9.953563,10.753325,10.210106,0.01,0,1.0,0.0,0.01,2.484073
2016-12-28,-0.04,-9.593441,-6.817204,-8.347459,-8.380038,-8.828062,-8.184177,-8.752170,-8.728730,-8.771433,...,-8.137694,10.130424,10.743454,10.169306,0.21,0,0.0,0.0,0.15,2.561096
